In [1]:
import pandas
import matplotlib.pyplot as plt
dataset = pandas.read_csv('/content/sample_data/testdata.csv', usecols=[0], engine='python')
plt.plot(dataset)
plt.show()

In [2]:
import numpy
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

Using TensorFlow backend.


In [3]:
numpy.random.seed(7)

In [4]:
# load the dataset
dataframe = pandas.read_csv('/content/sample_data/testdata.csv', usecols=[0], engine='python')
dataset = dataframe.values
dataset = dataset.astype('float32')

In [5]:
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)
len(dataset)

778

In [6]:
# split into train and test sets
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print(len(train), len(test))

521 257


In [7]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return numpy.array(dataX), numpy.array(dataY)

In [8]:
# reshape into X=t and Y=t+1
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [9]:
# reshape input to be [samples, time steps, features]
trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [10]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2)

Epoch 1/100
 - 1s - loss: 0.0225
Epoch 2/100
 - 1s - loss: 0.0083
Epoch 3/100
 - 1s - loss: 0.0037
Epoch 4/100
 - 1s - loss: 0.0012
Epoch 5/100
 - 1s - loss: 7.1498e-04
Epoch 6/100
 - 1s - loss: 6.8238e-04
Epoch 7/100
 - 1s - loss: 6.8413e-04
Epoch 8/100
 - 1s - loss: 6.8321e-04
Epoch 9/100
 - 1s - loss: 6.9061e-04
Epoch 10/100
 - 1s - loss: 6.9454e-04
Epoch 11/100
 - 1s - loss: 7.0276e-04
Epoch 12/100
 - 1s - loss: 6.7753e-04
Epoch 13/100
 - 1s - loss: 6.8040e-04
Epoch 14/100
 - 1s - loss: 6.9271e-04
Epoch 15/100
 - 1s - loss: 6.7488e-04
Epoch 16/100
 - 1s - loss: 6.9148e-04
Epoch 17/100
 - 1s - loss: 6.8342e-04
Epoch 18/100
 - 1s - loss: 7.0198e-04
Epoch 19/100
 - 1s - loss: 7.0988e-04
Epoch 20/100
 - 1s - loss: 6.9262e-04
Epoch 21/100
 - 1s - loss: 6.8566e-04
Epoch 22/100
 - 1s - loss: 6.8135e-04
Epoch 23/100
 - 1s - loss: 6.8374e-04
Epoch 24/100
 - 1s - loss: 6.9496e-04
Epoch 25/100
 - 1s - loss: 6.9488e-04
Epoch 26/100
 - 1s - loss: 6.9213e-04
Epoch 27/100
 - 1s - loss: 6.8008e-04

In [11]:
# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

Train Score: 9.75 RMSE
Test Score: 12.95 RMSE


In [12]:
# shift train predictions for plotting
trainPredictPlot = numpy.empty_like(dataset)
trainPredictPlot[:, :] = numpy.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
# shift test predictions for plotting
testPredictPlot = numpy.empty_like(dataset)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict
# plot baseline and predictions
plt.plot(scaler.inverse_transform(dataset))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()

In [13]:
testPredictPlot

array([[      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],
       [      nan],


In [16]:
len(testPredictPlot)

778